In [1]:
import numpy as np
import pandas as pd
import plotly

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
ticker1 = 'AOT.BK'
ticker2 = 'ACE.BK'
ticker3 = 'KEX.BK'
ticker4 = 'EPG.BK'
ticker5 = 'AAV.BK'
ticker6 = 'DELTA.BK'
query1 = "select * from stock_table_hr where `ticker` == '%s'" % ticker1
query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-04' and datetime < '2023-01-06'" % ticker2
query3 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker3
query4 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker4
query5 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2023-01-05' and datetime < '2023-01-06'" % ticker5
query6 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-12-20' and datetime < '2023-01-06'" % ticker6
AOT = pd.read_sql(query1,conn)
ACE = pd.read_sql(query2,conn)
KEX = pd.read_sql(query3,conn)
EPG = pd.read_sql(query4,conn)
AAV = pd.read_sql(query5,conn)
DELTA = pd.read_sql(query6,conn)
x = [ticker1,ticker2,ticker3,ticker4,ticker5,ticker6]
count = 0
#declare figure
fig = go.Figure()
DELTA['time'] = pd.to_datetime(DELTA['Datetime'])
DELTA.set_index(DELTA['time'],inplace = True)

#Candlestick
for column in [AOT,ACE,KEX,EPG,AAV,DELTA]:
    # name = column.Ticker[0]
    fig.add_trace(go.Candlestick(x=column.Datetime,
                    open=column.Open,
                    high=column.High,
                    low=column.Low,
                    close=column.Close, 
                    name = x[count],))
    count += 1
    
df_resample = DELTA.resample('H').max()
merged_index  = DELTA.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]
fig.update_xaxes( rangebreaks=[ dict(values = timegap, dvalue = 3600000) ])

# Add titles
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'Disable',
                method = 'update',
                args = [{'visible': [False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'Disable',
                        'showlegend':True}]),

            dict(label = 'AOT',
                method = 'update',
                args = [{'visible': [True, False, False, False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'AOT',
                        'showlegend':True}]),
            dict(label = 'ACE',
                method = 'update',
                args = [{'visible': [False, True, False, False, False, False]},
                        {'title': 'ACE',
                        'showlegend':True}]),
            dict(label = 'KEX',
                method = 'update',
                args = [{'visible': [False, False, True, False, False, False]},
                        {'title': 'KEX',
                        'showlegend':True}]),
            dict(label = 'EPG',
                method = 'update',
                args = [{'visible': [False, False, False, True, False, False]},
                        {'title': 'EPG',
                        'showlegend':True}]),
            dict(label = 'AAV',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False]},
                          {'title': 'AAV',
                           'showlegend':True}]),
            dict(label = 'DELTA',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True]},
                          {'title': 'DELTA',
                           'showlegend':True}]),
            ])
        )
    ])

fig.update_traces(visible="legendonly")
fig.update_yaxes(fixedrange=False)
# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Show
fig.show()

In [10]:
DELTA['time'] = pd.to_datetime(DELTA['Datetime'])
DELTA.set_index(DELTA['time'],inplace = True)
df_resample = DELTA.resample('H').max()
merged_index  = DELTA.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]
print(timegap)

DatetimeIndex(['2022-12-01 01:00:00', '2022-12-01 02:00:00',
               '2022-12-01 03:00:00', '2022-12-01 04:00:00',
               '2022-12-01 05:00:00', '2022-12-01 06:00:00',
               '2022-12-01 07:00:00', '2022-12-01 08:00:00',
               '2022-12-01 09:00:00', '2022-12-01 10:00:00',
               ...
               '2023-01-04 14:00:00', '2023-01-04 15:00:00',
               '2023-01-04 16:00:00', '2023-01-04 17:00:00',
               '2023-01-04 18:00:00', '2023-01-04 19:00:00',
               '2023-01-04 20:00:00', '2023-01-04 21:00:00',
               '2023-01-04 22:00:00', '2023-01-04 23:00:00'],
              dtype='datetime64[ns]', name='time', length=818, freq=None)


In [134]:
from datetime import timedelta
import yfinance as yf
import plotly.graph_objects as go

df = yf.download(tickers='DELTA.BK', period='10d', interval='1h')
fig = go.Figure()
fig.add_trace(go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close, name = 'market data'))
df['MA50'] = df['Close'].rolling(window = 50 , min_periods = 0).mean()
df['MA200'] = df['Close'].rolling(window = 200 , min_periods = 0).mean()
fig.add_trace(go.Scatter(name = 'MA50',x=df.index,y=df['MA50']))
fig.add_trace(go.Scatter(name = 'MA200',x=df.index,y=df['MA200']))
df_resample = df.resample('H').max()
merged_index  = df.index.append(df_resample.index)
timegap = merged_index[~merged_index.duplicated(keep = False)]

fig.update_xaxes( rangebreaks=[ dict(values = timegap, dvalue = 3600000) ])

fig.update_yaxes(fixedrange=False)
fig.show()
print(timegap)

[*********************100%***********************]  1 of 1 completed


DatetimeIndex(['2022-12-30 13:00:00', '2022-12-30 17:00:00',
               '2022-12-30 18:00:00', '2022-12-30 19:00:00',
               '2022-12-30 20:00:00', '2022-12-30 21:00:00',
               '2022-12-30 22:00:00', '2022-12-30 23:00:00',
               '2022-12-31 00:00:00', '2022-12-31 01:00:00',
               ...
               '2023-01-13 01:00:00', '2023-01-13 02:00:00',
               '2023-01-13 03:00:00', '2023-01-13 04:00:00',
               '2023-01-13 05:00:00', '2023-01-13 06:00:00',
               '2023-01-13 07:00:00', '2023-01-13 08:00:00',
               '2023-01-13 09:00:00', '2023-01-13 13:00:00'],
              dtype='datetime64[ns]', name='Datetime', length=283, freq=None)


In [139]:
import numpy as np
import pandas as pd
import plotly

#Data Source
import yfinance as yf

#Data viz
import plotly.graph_objs as go
import sqlite3

conn = sqlite3.connect("stock.sqlite")
cur = conn.cursor()
ticker = input("Enter tickers")
query1 = "select * from stock_table_hr where `ticker` == '%s' and datetime > '2023-01-01' and datetime < '2023-01-06'" % ticker
query2 = "select * from stock_table_d where `ticker` == '%s' and datetime > '2022-05-01' and datetime < '2023-01-06'" % ticker
query3 = "select * from stock_table_mo where `ticker` == '%s' and datetime > '2022-01-05' and datetime < '2023-01-06'" % ticker
Hr = pd.read_sql(query1,conn)
Day = pd.read_sql(query2,conn)
Mo = pd.read_sql(query3,conn)
#declare figure
fig = go.Figure()
# Day['time'] = pd.to_datetime(Day['Datetime'])
# Day.set_index(Day['time'],inplace = True)

#Candlestick
for column in [Hr,Day,Mo]:
    # name = column.Ticker[0]
    column['time'] = pd.to_datetime(column['Datetime'])
    column['MA50'] = column['Close'].rolling(window = 50 , min_periods = 0).mean()
    column['MA200'] = column['Close'].rolling(window = 200 , min_periods = 0).mean()
    column.set_index(column['time'],inplace = True)
    fig.add_trace(go.Candlestick(x=column.Datetime,
                    open=column.Open,
                    high=column.High,
                    low=column.Low,
                    close=column.Close, 
                    name = ticker,))
    fig.add_trace(go.Scatter(name = 'MA50',x=column.Datetime,y=column['MA50']))
    fig.add_trace(go.Scatter(name = 'MA200',x=column.Datetime,y=column['MA200']))
    
    
df_resample_hr = Hr.resample('H').max()
merged_index_hr  = Hr.index.append(df_resample_hr.index)
timegap_hr = merged_index_hr[~merged_index_hr.duplicated(keep = False)]

df_resample_day = Day.resample('D').max()
merged_index_day  = Day.index.append(df_resample_day.index)
timegap_day = merged_index_day[~merged_index_day.duplicated(keep = False)]

# fig.update_xaxes( rangebreaks=[ dict(values = timegap_hr , dvalue = 3600000)])

fig.update_xaxes(rangebreaks=[dict(values=timegap_day),
                              dict(values=timegap_hr, dvalue=3600000)])

fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'Disable',
                method = 'update',
                args = [{'visible': [False, False, False]}, # the index of True aligns with the indices of plot traces
                        {'title': 'Disable',
                        'showlegend':True}]),
            dict(label = 'Hour',
                method = 'update',
                args = [{'visible': [True, True, True, False, False, False, False, False, False]},
                        {'title': '%s Hour' % ticker,
                        'showlegend':True}]),
            dict(label = 'Day',
                method = 'update',
                args = [{'visible': [False, False, False, True, True, True, False, False, False]},
                        {'title': '%s Day' % ticker,
                        'showlegend':True}]),
            dict(label = 'Mount',
                method = 'update',
                args = [{'visible': [False, False, False, False, False, False, True, True, True]},
                        {'title': '%s Mount' % ticker,
                        'showlegend':True}]),
            ])
        )
    ])
# fig.update_yaxes(rangemode="nonnegative")
# fig.update_xaxes(rangemode="nonnegative")
fig.update_traces(visible="legendonly")
fig.update_yaxes(fixedrange=False)
fig.update_layout(hovermode = "x")
# fig.update_xaxes(
#         rangeslider_visible=True,
#         rangebreaks=[
#             # NOTE: Below values are bound (not single values), ie. hide x to y
#             dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
#             dict(bounds=[16, 9.5], pattern="hour"),  # hide hours outside of 9.30am-4pm
#             # dict(values=["2019-12-25", "2020-12-24"])  # hide holidays (Christmas and New Year's, etc)
#         ]
#     )

# X-Axes
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=15, label="15m", step="minute", stepmode="backward"),
            dict(count=45, label="45m", step="minute", stepmode="backward"),
            dict(count=1, label="HTD", step="hour", stepmode="todate"),
            dict(count=3, label="3h", step="hour", stepmode="backward"),
            dict(step="all")
        ])
    )
)

#Show
fig.show()

In [81]:
print(timegap_day)

DatetimeIndex(['2022-01-04 01:00:00', '2022-01-04 02:00:00',
               '2022-01-04 03:00:00', '2022-01-04 04:00:00',
               '2022-01-04 05:00:00', '2022-01-04 06:00:00',
               '2022-01-04 07:00:00', '2022-01-04 08:00:00',
               '2022-01-04 09:00:00', '2022-01-04 10:00:00',
               ...
               '2023-01-04 14:00:00', '2023-01-04 15:00:00',
               '2023-01-04 16:00:00', '2023-01-04 17:00:00',
               '2023-01-04 18:00:00', '2023-01-04 19:00:00',
               '2023-01-04 20:00:00', '2023-01-04 21:00:00',
               '2023-01-04 22:00:00', '2023-01-04 23:00:00'],
              dtype='datetime64[ns]', name='time', length=8541, freq=None)
